In [1]:
# this providing timing on every cell
!pip install ipython-autotime
%load_ext autotime

In [73]:
#-- IMPORT packages
import cudf
import cugraph
import numpy as np
from collections import OrderedDict
import time

time: 609 µs


In [79]:
start_time = time.time()

time: 506 µs


In [80]:
df = cudf.DataFrame()

time: 397 µs


In [81]:
# read a data file - this works for both Nodes and Edges
#import pandas as pd
def read_data(f, col, dt) :
    return cudf.read_csv(f, 
                         names=col, 
                         dtype=dt, 
                         delimiter=',', 
                         comment='#',
                         skiprows=1,
                         skip_blank_lines=True,
                         skipinitialspace=True
                        )

time: 672 µs


In [82]:
files = [
    '../data/800K_after_concat.csv',
#    '/content/drive/My Drive/colab data/400k_after_concat_data.csv',
#    '/content/drive/My Drive/colab data/25K_after_concat.csv'
]

time: 360 µs


In [83]:
start_time = time.time()

time: 330 µs


In [84]:
datafile = files[0]
raw_data = cudf.read_csv(datafile, delimiter=",", names=['node_1', 'node_2'], skiprows=1)
raw_data.rename(columns={'node_1':'src_str', 'node_2':'dst_str'}, inplace=True)

time: 134 ms


In [85]:
print(len(raw_data))

1441536
time: 448 µs


In [86]:
# Drop duplicates
raw_data = raw_data.drop_duplicates()

time: 118 ms


In [87]:
print(len(raw_data))

1441536
time: 437 µs


**Workaround for existing code as str.hash() is not working properly**

In [88]:
new_series=cudf.concat([raw_data['src_str'],raw_data['dst_str']])
temp = new_series.hash_values()
raw_data['src_hash'] = temp[:len(raw_data)]
raw_data['dst_hash'] = temp[len(raw_data):]

time: 95.5 ms


In [89]:
# Create vertex IDs
# The data has the nodes IDs as string, we need that to be intergers 
#raw_data['src_hash'] = raw_data['src_str'].str.hash()
#raw_data['dst_hash'] = raw_data['dst_str'].str.hash()

time: 270 µs


In [90]:
# Renumber the hash values to a smaller contiguous range 
raw_data['src'], raw_data['dst'], N = cugraph.renumber(raw_data['src_hash'], raw_data['dst_hash'])

time: 9.05 ms


In [91]:
graphtime = time.time()

time: 386 µs


In [92]:
### Now build the Graph
G = cugraph.Graph() # Graph() is undirected graph
G.add_edge_list(raw_data['src'], raw_data['dst'])

time: 18.8 ms


In [93]:
print("Gpu Graph")
print("\tNumber of Vertices: " + str(G.number_of_vertices()))
print("\tNumber of Edges:    " + str(G.number_of_edges()))

Gpu Graph
	Number of Vertices: 808800
	Number of Edges:    2563072
time: 1.27 ms


In [100]:
# get a list of vertices
wcc = cugraph.weakly_connected_components(G)
print ("Time taken to partition the skus : " + str(time.time() - graphtime) + " seconds")

Time taken to partition the skus : 431.94613575935364 seconds
time: 46.2 ms


In [103]:
wcc.index

RangeIndex(start=0, stop=808800)

time: 2.32 ms


In [95]:
#label_gby = wcc.groupby('labels')
#label_count = label_gby.count()

time: 225 µs


In [96]:
extract_time = time.time()
wcc['vertex'] = wcc.index

time: 1.41 ms


In [97]:
wcc.head()

,labels,vertices,vertex
0,1,0,0
1,2,1,1
2,3,2,2
3,4,3,3
4,5,4,4


time: 38.1 ms


In [98]:
#src_side = cudf.DataFrame()
#src_side['node']   = raw_data['src_str']
#src_side['vertex'] = raw_data['src'].astype(np.int64)

#dst_side = cudf.DataFrame()
#dst_side['node']   = raw_data['dst_str']
#dst_side['vertex'] = raw_data['dst'].astype(np.int64)

answer = cudf.DataFrame()
answer = cudf.concat([raw_data['src_str'], raw_datade])
answer = answer.drop_duplicates()

#del src_side
#del dst_side

NameError: name 'src_side' is not defined

time: 9.03 ms


In [99]:
a = wcc.merge(answer, on='vertex', how='left')
a = a.drop_duplicates()
a = a.sort_values(by='labels', ascending=True)

ValueError: 'vertex' is not in list

time: 15.9 ms


In [78]:
a.head()

,labels,vertices,vertex,node
51104,1,0,0,ESP_PSR_110COMP1_19@@ESP_PSR_15PLANT_19
51126,1,22,22,ESP_PSR_24FG_19@@ESP_PSR_13DC_19
10568,1,40,40,ESP_PSR_47RES_19@@dummy
78222,1,78,78,ESP_PSR_81FG_19@@ESP_PSR_19PLANT_19
36917,1,117,117,ESP_PSR_35FG_19@@ESP_PSR_12PLANT_19


time: 31.4 ms


In [ ]:
print ("Time taken to extract connected skus :" + str(time.time() - extract_time) + " seconds")
print ("Time taken from import csv to extract connected skus :" + str(time.time() - start_time) + " seconds")

Time taken to extract connected skus :0.6352226734161377 seconds
Time taken from import csv to extract connected skus :4.768831014633179 seconds
time: 2.22 ms


In [ ]:
print ("detected " + str( len(wcc['labels'].unique()) ) + " components")

detected 32 components
time: 536 ms


**NETWORKX RUN**

In [39]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

time: 528 µs


In [40]:
start_time = time.time()
datafile = files[0]
df = pd.read_csv(datafile, delimiter=",", names=['node_1', 'node_2'], skiprows=1)
df.rename(columns={'node_1':'src', 'node_2':'dst'}, inplace=True)

time: 1.35 s


In [41]:
graph_time = time.time()

time: 425 µs


In [42]:
cpuG=nx.from_pandas_edgelist(df, source='src', target='dst',create_using=nx.DiGraph)
msg = "Time taken to partition the skus  " + str(time.time() - graph_time) + "\n"
print(msg)

Time taken to partition the skus  6.696590185165405

time: 6.69 s


In [43]:
extract_time = time.time()
connectedskus = sorted(nx.weakly_connected_components(cpuG), key=len, reverse=True)
nodeslist = []
batchnum = 0
for consku in connectedskus:
  batchnum = batchnum + 1
  for val in consku:
          splitsku = val.split("@@")
          skuname = splitsku[0]
          locname = splitsku[1]
          t = (skuname, locname, batchnum)
          nodeslist.append(t)

msg = "Time taken to extract connected skus  " + str(time.time() - extract_time) + "\n"
print(msg)

Time taken to extract connected skus  3.211749315261841

time: 3.21 s


In [44]:
print ("Time taken from import csv to extract connected skus :" + str(time.time() - start_time) + " seconds")

Time taken from import csv to extract connected skus :11.274402379989624 seconds
time: 616 µs
